In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# -------------------------------
# Simulated EEG data (replace with your real data)
# -------------------------------
# Let's assume:
# - 1000 samples
# - 64 EEG channels
# - 128 time points per sample
# - 4 output classes
num_samples = 1000
num_channels = 64
sequence_length = 128
num_classes = 4

# Simulated EEG data
df = pd.read_csv('../data/preprocessed.csv')
X = df.drop(columns=['Unnamed: 0', '0'])
y = df['0']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# -------------------------------
# CNN + LSTM model
# -------------------------------
class CNN_LSTM(nn.Module):
    def __init__(self, num_channels, sequence_length, num_classes):
        super(CNN_LSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=128, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        # Calculate new sequence length after pooling
        self.lstm_input_size = 64
        self.seq_len_after_cnn = sequence_length // 4  # two maxpools

        self.lstm = nn.LSTM(input_size=self.lstm_input_size, hidden_size=64, num_layers=1, batch_first=True)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # x: [batch_size, channels, time_steps]
        x = self.pool1(self.relu1(self.conv1(x)))  # -> [B, 128, L/2]
        x = self.pool2(self.relu2(self.conv2(x)))  # -> [B, 64, L/4]

        # LSTM expects input [batch, seq_len, features]
        x = x.permute(0, 2, 1)  # -> [B, L/4, 64]
        lstm_out, _ = self.lstm(x)  # -> [B, L/4, 64]
        out = lstm_out[:, -1, :]  # take output of last time step

        out = self.fc(out)
        return out

# -------------------------------
# Training
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_LSTM(num_channels, sequence_length, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameterss(), lr=0.001)

n_epochs = 10
batch_size = 32

for epoch in range(n_epochs):
    model.train()
    permutation = torch.randperm(X_train.size(0))

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X_train[indices].to(device), y_train[indices].to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")

# -------------------------------
# Evaluation
# -------------------------------
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device))
    y_pred_labels = torch.argmax(y_pred, dim=1).cpu().numpy()
    acc = accuracy_score(y_test.numpy(), y_pred_labels)
    print("Test Accuracy:", acc)


AttributeError: module 'torch._functorch.eager_transforms' has no attribute 'grad_and_value'